In [24]:
import pandas as pd

In [25]:
# 1. Load the data
df = pd.read_csv('../data/mindx test dataset.csv')

In [26]:
# 2. Define "Efficiency" (Fuel burned per Nautical Mile)
# Formula: Efficiency = Fuel / Distance
df['fuel_per_mile'] = df['fuel_consumption'] / df['distance']

In [27]:
# Global Baseline (The "Average for all voyages")
fleet_avg_value = df['fuel_per_mile'].mean()

In [28]:
# Analysis 1: The "Acute" Anomaly (Bad Voyage)
# Filter for Calm weather to isolate mechanical/physical issues
calm_df = df[df['weather_conditions'] == 'Calm']
worst_voyage = calm_df.sort_values(by='fuel_per_mile', ascending=False).iloc[0]

In [29]:
print(f"Acute Anomaly Report (Single Voyage)")
print(f"Vessel: {worst_voyage['ship_id']}")
print(f"Route: {worst_voyage['route_id']}")
print(f"Weather: {worst_voyage['weather_conditions']}")
print("-" * 30)
print(f"Worst Voyage Value: {worst_voyage['fuel_per_mile']:.2f} kg/nm")
print(f"All Voyages Average: {fleet_avg_value:.2f} kg/nm")
print(f"Comparison: Voyage is +{((worst_voyage['fuel_per_mile'] - fleet_avg_value)/fleet_avg_value)*100:.1f}% worse than fleet average")
print("-" * 30)

Acute Anomaly Report (Single Voyage)
Vessel: NG117
Route: Lagos-Apapa
Weather: Calm
------------------------------
Worst Voyage Value: 49.91 kg/nm
All Voyages Average: 28.64 kg/nm
Comparison: Voyage is +74.2% worse than fleet average
------------------------------


In [30]:
# ANALYSIS 2: The "Aggregate" Anomaly (Bad Ship)
# Group by Ship ID and calculate average efficiency
ship_performance = df.groupby('ship_id')['fuel_per_mile'].mean().reset_index()
worst_ship_overall = ship_performance.sort_values(by='fuel_per_mile', ascending=False).iloc[0]

In [31]:
print(f"Chronic Anomaly Report (Overall Ship)")
print(f"Vessel: {worst_ship_overall['ship_id']}")
print(f"Ship Avg Value: {worst_ship_overall['fuel_per_mile']:.2f} kg/nm")
print(f"All Ships Average: {fleet_avg_value:.2f} kg/nm")
print(f"Comparison: Ship is +{((worst_ship_overall['fuel_per_mile'] - fleet_avg_value)/fleet_avg_value)*100:.1f}% worse than fleet average")

Chronic Anomaly Report (Overall Ship)
Vessel: NG108
Ship Avg Value: 43.53 kg/nm
All Ships Average: 28.64 kg/nm
Comparison: Ship is +52.0% worse than fleet average
